In [2]:
import tkinter as tk
from tkinter import filedialog
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np

# 加载预训练的BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

def extract_keywords():
    # 打开文件对话框，让用户选择要上传的文件
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    
    # 读取用户选择的文件内容
    with open(file_path, 'r') as file:
        text = file.read()
    
    # 将文本数据转换为适合BERT模型输入的格式
    inputs = tokenizer(text, return_tensors='tf')

    # 输入文本数据到BERT模型中，获取模型的输出
    outputs = model(inputs)

    # 获取模型的序列输出
    last_hidden_state = outputs.last_hidden_state

    # 计算每个输入文本的文本向量，即将其序列输出中的每个词向量相加
    text_vectors = tf.reduce_sum(last_hidden_state, axis=1)

    # 计算文本向量之间的余弦相似度
    text_vectors_transpose = tf.transpose(text_vectors)
    similarities = tf.matmul(text_vectors, text_vectors_transpose) / (tf.norm(text_vectors, axis=1, keepdims=True) * tf.norm(text_vectors_transpose, axis=0, keepdims=True))

    # 找到与其他文本最不相似的文本索引
    least_similar_index = np.argmin(np.sum(similarities, axis=0))

    # 提取关键词
    keywords = tokenizer.convert_ids_to_tokens(inputs['input_ids'][least_similar_index].numpy())

    # 将提取的关键词结果显示在文本框中
    result_text.delete('1.0', tk.END)
    result_text.insert(tk.END, f"Keywords: {' '.join(keywords)}")

# 创建主窗口
root = tk.Tk()
root.title("关键词提取工具")

# 创建上传按钮
upload_button = tk.Button(root, text="上传文件", command=extract_keywords)
upload_button.pack(pady=10)

# 创建显示结果的文本框
result_text = tk.Text(root, height=20, width=50)
result_text.pack()

# 运行主循环
root.mainloop()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [3]:
import tkinter as tk
from tkinter import filedialog
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

# 加载预训练的BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

def extract_keywords():
    # 打开文件对话框，让用户选择要上传的文件
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    
    # 读取用户选择的文件内容
    with open(file_path, 'r') as file:
        text = file.read()
    
    # 将文本数据转换为适合BERT模型输入的格式
    inputs = tokenizer(text, return_tensors='tf')

    # 输入文本数据到BERT模型中，获取模型的输出
    outputs = model(inputs)

    # 获取模型的序列输出
    sequence_output = outputs.last_hidden_state

    # 计算每个词语的范数作为重要性分数
    word_scores = tf.norm(sequence_output, axis=-1)

    # 获取每个词语的重要性分数
    word_scores = word_scores.numpy()[0]

    # 获取词语列表
    word_list = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].numpy())

    # 构建关键词-重要性分数字典
    keyword_scores = {word: score for word, score in zip(word_list, word_scores)}

    # 按照重要性分数降序排列关键词
    sorted_keywords = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)

    # 将提取的关键词结果显示在文本框中
    result_text.delete('1.0', tk.END)
    for keyword, score in sorted_keywords:
        result_text.insert(tk.END, f"{keyword}: {score}\n")

# 创建主窗口
root = tk.Tk()
root.title("关键词提取工具")

# 创建上传按钮
upload_button = tk.Button(root, text="上传文件", command=extract_keywords)
upload_button.pack(pady=10)

# 创建显示结果的文本框
result_text = tk.Text(root, height=20, width=50)
result_text.pack()

# 运行主循环
root.mainloop()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w